# Frequency Modeling using Generalized Linear Models

**Project:** PRISM – Predictive & Research-based Insurance Statistical Modeling

## Objective
To model claim frequency using Poisson and Negative Binomial GLMs with exposure offsets and compare their suitability.


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
freq = pd.read_csv(
    "/content/drive/MyDrive/freMTPL2freq.csv"
)

freq = freq.rename(columns={
    "IDpol": "policy_id",
    "ClaimNb": "claim_count",
    "Exposure": "exposure",
    "Area": "area",
    "VehPower": "vehicle_power",
    "VehAge": "vehicle_age",
    "DrivAge": "driver_age",
    "BonusMalus": "bonus_malus",
    "VehBrand": "vehicle_brand",
    "VehGas": "vehicle_gas"
})

In [5]:
freq.head()


,policy_id,claim_count,exposure,area,vehicle_power,vehicle_age,driver_age,bonus_malus,vehicle_brand,vehicle_gas,Density,Region
0,1.0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,3.0,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,5.0,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,10.0,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,11.0,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


In [6]:
formula = """
claim_count ~ C(area) + vehicle_power + vehicle_age + driver_age + bonus_malus + C(vehicle_brand) + C(vehicle_gas)
"""

poisson_model = smf.glm(
    formula=formula,
    data=freq,
    family=sm.families.Poisson(),
    offset=np.log(freq["exposure"])
).fit()

poisson_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            claim_count   No. Observations:               678013
Model:                            GLM   Df Residuals:                   677992
Model Family:                 Poisson   Df Model:                           20
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4340e+05
Date:                Sat, 17 Jan 2026   Deviance:                   2.1747e+05
Time:                        11:13:44   Pearson chi2:                 1.79e+06
No. Iterations:                     7   Pseudo R-squ. (CS):            0.01028
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -3.9326      0.041    -96.171      0.000      -4.013      -3.852
C(area)[T.B]                  0.0430      0.021      2.004      0.045       0.001       0.085
C(area)[T.C]                  0.0735      0.017      4.231      0.000       0.039       0.108
C(area)[T.D]                  0.1541      0.018      8.571      0.000       0.119       0.189
C(area)[T.E]                  0.1969      0.018     10.681      0.000       0.161       0.233
C(area)[T.F]                  0.1727      0.034      5.147      0.000       0.107       0.239
C(vehicle_brand)[T.B10]      -0.0104      0.037     -0.282      0.778      -0.082       0.062
C(vehicle_brand)[T.B11]       0.0687      0.040      1.736      0.082      -0.009       0.146
C(vehicle_brand)[T.B12]       0.1222      0.017      7.327      0.000       0.090       0.155
C(vehicle_brand)[T.B13]       0.0245      0.041      0.599      0.549      -0.056       0.105
C(vehicle_brand)[T.B14]      -0.1699      0.078     -2.167      0.030      -0.324      -0.016
C(vehicle_brand)[T.B2]       -0.0168      0.015     -1.099      0.272      -0.047       0.013
C(vehicle_brand)[T.B3]       -0.0109      0.022     -0.497      0.619      -0.054       0.032
C(vehicle_brand)[T.B4]       -0.0467      0.030     -1.572      0.116      -0.105       0.012
C(vehicle_brand)[T.B5]        0.0550      0.025      2.223      0.026       0.007       0.103
C(vehicle_brand)[T.B6]       -0.0533      0.028     -1.881      0.060      -0.109       0.002
C(vehicle_gas)[T.Regular]     0.0545      0.011      4.985      0.000       0.033       0.076
vehicle_power                 0.0145      0.003      5.255      0.000       0.009       0.020
vehicle_age                  -0.0384      0.001    -33.015      0.000      -0.041      -0.036
driver_age                    0.0066      0.000     16.780      0.000       0.006       0.007
bonus_malus                   0.0225      0.000     73.165      0.000       0.022       0.023
=============================================================================================
"""

In [7]:
poisson_model.deviance / poisson_model.df_resid


np.float64(0.320760605713998)

## Poisson GLM Results

The Poisson model estimates claim frequency using rating factors with an exposure offset.  
The deviance-to-degrees-of-freedom ratio is below 1, indicating no evidence of overdispersion.  
This suggests that the Poisson model is an appropriate baseline for claim frequency modeling.




## Negative Binomial GLM

The Negative Binomial model is fitted to relax the Poisson assumption and account for potential overdispersion in claim frequency.




In [8]:
nb_model = smf.glm(
    formula=formula,
    data=freq,
    family=sm.families.NegativeBinomial(),
    offset=np.log(freq["exposure"])
).fit()

nb_model.summary()

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            claim_count   No. Observations:               678013
Model:                            GLM   Df Residuals:                   677992
Model Family:        NegativeBinomial   Df Model:                           20
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4298e+05
Date:                Sat, 17 Jan 2026   Deviance:                   1.8946e+05
Time:                        11:27:05   Pearson chi2:                 1.73e+06
No. Iterations:                     8   Pseudo R-squ. (CS):           0.009682
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -3.9392      0.043    -91.332      0.000      -4.024      -3.855
C(area)[T.B]                  0.0434      0.022      1.959      0.050    -2.4e-05       0.087
C(area)[T.C]                  0.0735      0.018      4.099      0.000       0.038       0.109
C(area)[T.D]                  0.1559      0.019      8.387      0.000       0.119       0.192
C(area)[T.E]                  0.1972      0.019     10.334      0.000       0.160       0.235
C(area)[T.F]                  0.1776      0.035      5.094      0.000       0.109       0.246
C(vehicle_brand)[T.B10]      -0.0069      0.038     -0.181      0.856      -0.082       0.068
C(vehicle_brand)[T.B11]       0.0690      0.041      1.677      0.093      -0.012       0.150
C(vehicle_brand)[T.B12]       0.1402      0.017      8.118      0.000       0.106       0.174
C(vehicle_brand)[T.B13]       0.0222      0.043      0.521      0.602      -0.061       0.106
C(vehicle_brand)[T.B14]      -0.1727      0.081     -2.136      0.033      -0.331      -0.014
C(vehicle_brand)[T.B2]       -0.0171      0.016     -1.078      0.281      -0.048       0.014
C(vehicle_brand)[T.B3]       -0.0111      0.023     -0.490      0.624      -0.056       0.033
C(vehicle_brand)[T.B4]       -0.0471      0.031     -1.530      0.126      -0.108       0.013
C(vehicle_brand)[T.B5]        0.0580      0.026      2.253      0.024       0.008       0.108
C(vehicle_brand)[T.B6]       -0.0535      0.029     -1.821      0.069      -0.111       0.004
C(vehicle_gas)[T.Regular]     0.0609      0.011      5.376      0.000       0.039       0.083
vehicle_power                 0.0136      0.003      4.783      0.000       0.008       0.019
vehicle_age                  -0.0391      0.001    -32.503      0.000      -0.041      -0.037
driver_age                    0.0066      0.000     16.116      0.000       0.006       0.007
bonus_malus                   0.0227      0.000     68.350      0.000       0.022       0.023
=============================================================================================
"""

In [9]:
poisson_model.aic, nb_model.aic


(np.float64(286838.15101673116), np.float64(285993.22311047727))

## Model Comparison

The Poisson and Negative Binomial models are compared using AIC.  
Although overdispersion was not strongly indicated, the Negative Binomial model achieves a lower AIC, suggesting a better fit.  
Therefore, the Negative Binomial GLM is selected as the final frequency model for pricing.
